In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl
from matplotlib.ticker import AutoLocator, AutoMinorLocator, LogLocator
from matplotlib.colors import Normalize
import glob
from scipy.interpolate import griddata
from pathlib import Path
import h5py
import sys
from matplotlib.lines import Line2D
from matplotlib.legend_handler import HandlerTuple

# Where am I running?
try:
    # Normal script
    here = Path(__file__).resolve().parent
except NameError:
    # Notebook / REPL
    here = Path.cwd()

phys_const_path = (here / '..' / 'phys_const').resolve()
sys.path.append(str(phys_const_path))

nsm_plots_path = (here / '..' / 'nsm_plots').resolve()
sys.path.append(str(nsm_plots_path))

nsm_plots_postproc = (here / '..' / 'nsm_instabilities').resolve()
sys.path.append(str(nsm_plots_postproc))

import phys_const as pc
import plot_functions as pf
import functions_angular_crossings as fac

### Reading hdf5 data

In [ ]:
dir_last = '/home/erick/software/devscrpts/gw170817_paper_plots/nsmcell_15_48_15_doma_48-1-1cm_cells_192-1-1_att_1_dt_1e-12s_novac_nomat_sinus_plt1500000_particles_angular_distributions.h5'

with h5py.File(dir_last, "r") as hf:

    # Read ELN angular distribution
    phi_eln_xln_fi_last = hf["phi_eln_xln_fi"][:]
    mu_eln_xln_fi_last = hf["mu_eln_xln_fi"][:]
    G_eln_xln_fi_last = hf["G_eln_xln_fi"][:]
    phi_eln_xln_fp_last = hf["phi_eln_xln_fp"][:]
    mu_eln_xln_fp_last = hf["mu_eln_xln_fp"][:]
    G_eln_xln_fp_last = hf["G_eln_xln_fp"][:]

    # Read e (nu_e) angular distribution
    phi_e_fi_last = hf["phi_e_fi"][:]
    mu_e_fi_last = hf["mu_e_fi"][:]
    G_e_fi_last = hf["G_e_fi"][:]
    phi_e_fp_last = hf["phi_e_fp"][:]
    mu_e_fp_last = hf["mu_e_fp"][:]
    G_e_fp_last = hf["G_e_fp"][:]

    # Read ebar (anti-nu_e) angular distribution
    phi_ebar_fi_last = hf["phi_ebar_fi"][:]
    mu_ebar_fi_last = hf["mu_ebar_fi"][:]
    G_ebar_fi_last = hf["G_ebar_fi"][:]
    phi_ebar_fp_last = hf["phi_ebar_fp"][:]
    mu_ebar_fp_last = hf["mu_ebar_fp"][:]
    G_ebar_fp_last = hf["G_ebar_fp"][:]

    # Read u (nu_x) angular distribution
    phi_u_fi_last = hf["phi_u_fi"][:]
    mu_u_fi_last = hf["mu_u_fi"][:]
    G_u_fi_last = hf["G_u_fi"][:]
    phi_u_fp_last = hf["phi_u_fp"][:]
    mu_u_fp_last = hf["mu_u_fp"][:]
    G_u_fp_last = hf["G_u_fp"][:]

dir_init = '/home/erick/software/devscrpts/gw170817_paper_plots/nsmcell_15_48_15_doma_48-1-1cm_cells_192-1-1_att_1_dt_1e-12s_novac_nomat_sinus_plt00000_particles_angular_distributions.h5'

with h5py.File(dir_init, "r") as hf:

    # Read ELN angular distribution
    phi_eln_xln_fi_init = hf["phi_eln_xln_fi"][:]
    mu_eln_xln_fi_init = hf["mu_eln_xln_fi"][:]
    G_eln_xln_fi_init = hf["G_eln_xln_fi"][:]
    phi_eln_xln_fp_init = hf["phi_eln_xln_fp"][:]
    mu_eln_xln_fp_init = hf["mu_eln_xln_fp"][:]
    G_eln_xln_fp_init = hf["G_eln_xln_fp"][:]

    # Read e (nu_e) angular distribution
    phi_e_fi_init = hf["phi_e_fi"][:]
    mu_e_fi_init = hf["mu_e_fi"][:]
    G_e_fi_init = hf["G_e_fi"][:]
    phi_e_fp_init = hf["phi_e_fp"][:]
    mu_e_fp_init = hf["mu_e_fp"][:]
    G_e_fp_init = hf["G_e_fp"][:]

    # Read ebar (anti-nu_e) angular distribution
    phi_ebar_fi_init = hf["phi_ebar_fi"][:]
    mu_ebar_fi_init = hf["mu_ebar_fi"][:]
    G_ebar_fi_init = hf["G_ebar_fi"][:]
    phi_ebar_fp_init = hf["phi_ebar_fp"][:]
    mu_ebar_fp_init = hf["mu_ebar_fp"][:]
    G_ebar_fp_init = hf["G_ebar_fp"][:]

    # Read u (nu_x) angular distribution
    phi_u_fi_init = hf["phi_u_fi"][:]
    mu_u_fi_init = hf["mu_u_fi"][:]
    G_u_fi_init = hf["G_u_fi"][:]
    phi_u_fp_init = hf["phi_u_fp"][:]
    mu_u_fp_init = hf["mu_u_fp"][:]
    G_u_fp_init = hf["G_u_fp"][:]

In [ ]:
def plot_colored_lines_grid(y_list, color_maps, titles, filename,
                            ylowerlimit=None, yupperlimit=None):
    fig, axs = plt.subplots(2, 4, figsize=(18, 6),
                            sharex=True, sharey=True,
                            gridspec_kw=dict(hspace=0.0, wspace=0.0))
    axs = axs.flatten()

    for i, y in enumerate(y_list):

        ax = axs[i]

        c = ax.pcolormesh(y[3], y[4], y[5], shading='auto', cmap=color_maps[i], vmin=ylowerlimit[i], vmax=yupperlimit[i])
        ax.scatter(y[0], y[1], c=y[2], cmap=color_maps[i], vmin=ylowerlimit[i], vmax=yupperlimit[i], s=8, edgecolor='gray')

        # ax.text(0.03, 0.15, titles[i], transform=ax.transAxes, fontsize=22,
        #     verticalalignment='top', horizontalalignment='left',
        #     bbox=dict(facecolor='none', alpha=0.7, edgecolor='none'))

        # Set horizontal axis in angles from 0 to 2pi, showing integers of pi and half fractions
        ax.set_xlim(0, 2 * np.pi)
        ax.set_xticks([0.5 * np.pi, 1.0 * np.pi, 1.5 * np.pi])
        ax.set_xticklabels([r'$\frac{1}{2}\pi$', r'$\pi$', r'$\frac{3}{2}\pi$'])
        ax.set_ylim(-1,1)
        ax.tick_params(axis='both', which='both', direction='in', top=True, right=True)
        ax.xaxis.set_minor_locator(AutoMinorLocator())
        ax.yaxis.set_minor_locator(AutoMinorLocator())
    
    # Shared axis labels
    fig.text(0.5, 0.02, r'$\phi$', ha='center', fontsize=22)
    fig.text(0.07, 0.5, r'$\cos \theta$', va='center', rotation='vertical', fontsize=22)

    # --- Colorbars per column, on top with ticks/labels above ---
    for col in range(4):  # three columns

        norm = Normalize(ylowerlimit[col], vmax=yupperlimit[col])
        cmap = plt.cm.get_cmap(color_maps[col])
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        # Position: [left, bottom, width, height] in figure coords
        left = 0.128 + col * (0.188+0.005)
        bottom = 0.92  # put high at top
        width = 0.188
        height = 0.03
        cax = fig.add_axes([left, bottom, width, height])
        cbar = fig.colorbar(sm, cax=cax, orientation='horizontal')

        # Put ticks + label above
        cbar.ax.xaxis.set_ticks_position('top')
        cbar.ax.xaxis.set_label_position('top')
        cbar.set_label(titles[col], fontsize=18, labelpad=8)
        cbar.ax.xaxis.set_minor_locator(AutoMinorLocator())

    # Save and show
    if filename:
        plt.savefig(filename, bbox_inches='tight')
    plt.show()
    plt.close(fig)

a=0
b=-485
c=None

plot_colored_lines_grid(
    y_list = [
        [phi_eln_xln_fi_init, mu_eln_xln_fi_init, G_eln_xln_fi_init/1e9, phi_eln_xln_fp_init, mu_eln_xln_fp_init, G_eln_xln_fp_init/1e9],
        [phi_e_fi_init, mu_e_fi_init,             G_e_fi_init      /1e9, phi_e_fp_init, mu_e_fp_init,             G_e_fp_init      /1e9],
        [phi_ebar_fi_init, mu_ebar_fi_init,       G_ebar_fi_init   /1e9, phi_ebar_fp_init, mu_ebar_fp_init,       G_ebar_fp_init   /1e9],
        [phi_u_fi_init, mu_u_fi_init,             G_u_fi_init      /1e9, phi_u_fp_init, mu_u_fp_init,             G_u_fp_init     /1e9],
        [phi_eln_xln_fi_last, mu_eln_xln_fi_last, G_eln_xln_fi_last/1e9, phi_eln_xln_fp_last, mu_eln_xln_fp_last, G_eln_xln_fp_last/1e9],
        [phi_e_fi_last, mu_e_fi_last,             G_e_fi_last      /1e9, phi_e_fp_last, mu_e_fp_last,             G_e_fp_last     /1e9],
        [phi_ebar_fi_last, mu_ebar_fi_last,       G_ebar_fi_last   /1e9, phi_ebar_fp_last, mu_ebar_fp_last,       G_ebar_fp_last  /1e9],
        [phi_u_fi_last, mu_u_fi_last,             G_u_fi_last      /1e9, phi_u_fp_last, mu_u_fp_last,             G_u_fp_last     /1e9],
    ],
    color_maps=[
        'seismic', 'Reds', 'Blues', 'Greens',
        'seismic', 'Reds', 'Blues', 'Greens',],
    titles = [
        r'$G_{\mathrm{ELN-XLN}}\,\left[10^9\,\mathrm{s}^{-1}\,\mathrm{Sr}^{-1}\right]$', 
        r'$G_{\nu_e}\,\left[10^9\,\mathrm{s}^{-1}\,\mathrm{Sr}^{-1}\right]$', 
        r'$G_{\bar{\nu}_e}\,\left[10^9\,\mathrm{s}^{-1}\,\mathrm{Sr}^{-1}\right]$', 
        r'$G_{\nu_\mu}\,\left[10^9\,\mathrm{s}^{-1}\,\mathrm{Sr}^{-1}\right]$', 
    ],
    filename = f"plots/angular_distribution.png",
    ylowerlimit = np.array([-4.51833597e+09,1.46257479e+09,6.71444161e+08,1.86834413e+06,-4.51833597e+09,1.46257479e+09,6.71444161e+08,1.86834413e+06])/1e9,
    yupperlimit = np.array([+4.51833597e+09,9.52227218e+09,1.21471491e+10,+4.36328775e+09,+4.51833597e+09,9.52227218e+09,1.21471491e+10,+4.36328775e+09])/1e9
)

### Spatial flavor structure plots

In [ ]:
with h5py.File(f"/home/erick/software/devscrpts/gw170817_paper_plots/nsmcell_15_48_15_doma_48-1-1cm_cells_192-1-1_att_1_dt_1e-12s_novac_nomat_sinus_n_spatial_flavor_evolution.h5", "r") as hf:
    time_s = hf["time_s"][:]
    x_cell_centers = hf["x_cell_centers_cm"][:]
    nee = hf["n_ee_inv_ccm"][:]
    nuu = hf["n_uu_inv_ccm"][:]
    ntt = hf["n_tt_inv_ccm"][:]
    neu = hf["n_eu_inv_ccm"][:]
    net = hf["n_et_inv_ccm"][:]
    nut = hf["n_ut_inv_ccm"][:]

print("nee shape:", nee.shape)
print("nuu shape:", nuu.shape)
print("ntt shape:", ntt.shape)
print("neu shape:", neu.shape)
print("net shape:", net.shape)
print("nut shape:", nut.shape)
print("time_s shape:", time_s.shape)
print("x_cell_centers shape:", x_cell_centers.shape)

In [ ]:
def plot_colored_lines_grid(x, y_list, time_s, titles, suptitle, filename, ylowerlimit=None, yupperlimit=None):
    fig, axs = plt.subplots(2, 3, figsize=(22, 8), sharex=True, sharey=False, gridspec_kw=dict(hspace=0.1, wspace=0.15))
    axs = axs.flatten()

    norm = Normalize(time_s.min(), time_s.max())
    cmap = plt.cm.winter

    for i, y in enumerate(y_list):
        ax = axs[i]
        for j in range(y.shape[0]):
            color = cmap(norm(time_s[j]))
            ax.plot(x, y[j, :], color=color)

        # Place the title inside the plot, top left corner
        ax.text(0.50, 0.50, titles[i], transform=ax.transAxes, fontsize=22,
            verticalalignment='top', horizontalalignment='left', bbox=dict(facecolor='None', alpha=1.0, edgecolor='none'))
        # ax.set_xlim(x.min(), x.max())
        # ax.set_ylim(ylowerlimit, yupperlimit)
        ax.tick_params(axis='both', which='both', direction='in', top=True, right=True)
        ax.xaxis.set_minor_locator(AutoMinorLocator())
        ax.yaxis.set_minor_locator(AutoMinorLocator())
        # if i % 3 == 0:
        #     ax.set_ylabel(r'$\log(n\, [\mathrm{cm}^{-3}])$')
        # if i >= 3:
        #     ax.set_xlabel(r'$\log(E\, [\mathrm{MeV}])$')

    fig.text(0.5, 0.02, r'$x\, [\mathrm{cm}] $', ha='center', fontsize=22)
    fig.text(0.07, 0.5, r'$n\, [10^{31}\,\mathrm{cm}^{-3}] $', va='center', rotation='vertical', fontsize=22)

    # Shared colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=axs, location='right', shrink=1.0, pad=0.01)
    cbar.set_label(r'$t \, [10^{-6}\,\mathrm{s}]$', fontsize=22)
    cbar.ax.yaxis.set_minor_locator(AutoMinorLocator())

    # Super title
    fig.suptitle(suptitle, fontsize=22)

    # Save and show
    if filename:
        plt.savefig(filename, bbox_inches='tight')
    plt.show()
    plt.close(fig)

a=0
b=200
c=2

plot_colored_lines_grid(
    x = x_cell_centers,
    y_list = [
        nee[a:b:c]/1e31,
        nuu[a:b:c]/1e31,
        ntt[a:b:c]/1e31,
        neu[a:b:c]/1e31,
        net[a:b:c]/1e31,
        nut[a:b:c]/1e31,
        # nee[a:b:c]-nee[0,:],
        # nuu[a:b:c]-nuu[0,:],
        # ntt[a:b:c]-ntt[0,:],
        # neu[a:b:c]-neu[0,:],
        # net[a:b:c]-net[0,:],
        # nut[a:b:c]-nut[0,:]
        # np.log10(nee[a:b:c]),
        # np.log10(nuu[a:b:c]),
        # np.log10(ntt[a:b:c]),
        # np.log10(neu[a:b:c]),
        # np.log10(net[a:b:c]),
        # np.log10(nut[a:b:c])
    ],
    time_s = time_s[a:b:c]/1e-6,
    titles = [
        r'$ee$', r'$\mu\mu$', r'$\tau\tau$',
        r'$e\mu$', r'$e\mu$', r'$\mu\tau$'
    ],
    suptitle = '',
    filename = f"plots/n_space.png",
    ylowerlimit = None,
    yupperlimit = None
)

### Furier modes

In [ ]:
with h5py.File(f"/home/erick/software/devscrpts/gw170817_paper_plots/nsmcell_15_48_15_doma_48-1-1cm_cells_192-1-1_att_1_dt_1e-12s_novac_nomat_sinus_n_furier_evolution.h5", "r") as hf:
    time_fft = hf["time_s"][:]
    lambda_cm_ee = hf["lambda_cm_ee"][:]
    f_lambda_ee = hf["f_lambda_ee"][:]
    lambda_cm_uu = hf["lambda_cm_uu"][:]
    f_lambda_uu = hf["f_lambda_uu"][:]
    lambda_cm_tt = hf["lambda_cm_tt"][:]
    f_lambda_tt = hf["f_lambda_tt"][:]
    lambda_cm_eu = hf["lambda_cm_eu"][:]
    f_lambda_eu = hf["f_lambda_eu"][:]
    lambda_cm_et = hf["lambda_cm_et"][:]
    f_lambda_et = hf["f_lambda_et"][:]
    lambda_cm_ut = hf["lambda_cm_ut"][:]
    f_lambda_ut = hf["f_lambda_ut"][:]


In [ ]:
def plot_colored_lines_grid(x, y_list, time_s, titles, suptitle, filename, ylowerlimit=None, yupperlimit=None):
    fig, axs = plt.subplots(2, 3, figsize=(22, 8), sharex=True, sharey=True, gridspec_kw=dict(hspace=0.0, wspace=0.0))
    axs = axs.flatten()

    norm = Normalize(time_s.min(), time_s.max())
    cmap = plt.cm.winter

    for i, y in enumerate(y_list):
        ax = axs[i]
        for j in range(y[1].shape[0]):
            color = cmap(norm(time_s[j]))
            ax.plot(2*np.pi/y[0][j,:], y[1][j, :], color=color)

        # Place the title inside the plot, top left corner
        ax.text(0.1, 0.9, titles[i], transform=ax.transAxes, fontsize=22,
            verticalalignment='top', horizontalalignment='left', bbox=dict(facecolor='None', alpha=1.0, edgecolor='none'))
        # ax.set_xlim(x.min(), x.max())
        # ax.set_ylim(ylowerlimit, yupperlimit)
        ax.tick_params(axis='both', which='both', direction='in', top=True, right=True)
        ax.xaxis.set_minor_locator(AutoMinorLocator())
        ax.yaxis.set_minor_locator(AutoMinorLocator())
        # if i % 3 == 0:
        #     ax.set_ylabel(r'$\log(n\, [\mathrm{cm}^{-3}])$')
        # if i >= 3:
        #     ax.set_xlabel(r'$\log(E\, [\mathrm{MeV}])$')

    fig.text(0.5, 0.02, r'$k\, [\mathrm{cm}^{-1}] $', ha='center', fontsize=22)
    fig.text(0.08, 0.5, r'$\log(\,\mathcal{F}\,\{n\} \, [\mathrm{cm}^{-2}]\,)$', va='center', rotation='vertical', fontsize=22)

    # Shared colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=axs, location='right', shrink=1.0, pad=0.01)
    cbar.set_label(r'$t \, [10^{-6}\,\mathrm{s}]$', fontsize=22)
    cbar.ax.yaxis.set_minor_locator(AutoMinorLocator())

    # Super title
    fig.suptitle(suptitle, fontsize=22)

    # Save and show
    if filename:
        plt.savefig(filename, bbox_inches='tight')
    plt.show()
    plt.close(fig)

a=0
b=120
c=2

plot_colored_lines_grid(
    x = lambda_cm_ee,
    y_list = [
        [lambda_cm_ee[a:b:c], np.log10(f_lambda_ee[a:b:c])],
        [lambda_cm_uu[a:b:c], np.log10(f_lambda_uu[a:b:c])],
        [lambda_cm_tt[a:b:c], np.log10(f_lambda_tt[a:b:c])],
        [lambda_cm_eu[a:b:c], np.log10(f_lambda_eu[a:b:c])],
        [lambda_cm_et[a:b:c], np.log10(f_lambda_et[a:b:c])],
        [lambda_cm_ut[a:b:c], np.log10(f_lambda_ut[a:b:c])]
    ],
    time_s = time_fft[a:b:c]/1e-6,
    titles = [
        r'$ee$', r'$\mu\mu$', r'$\tau\tau$',
        r'$e\mu$', r'$e\tau$', r'$\mu\tau$'
    ],
    suptitle = '',
    filename = f"plots/n_furier.png",
    ylowerlimit = None,
    yupperlimit = None
)